<img src="graphics/header.png" width="75%"/>

# **Machine Learning for Clinical Care**
---

In this module, we will work through an illustrated example of using machine learning (ML) to develop a patient risk prediction tool using electronic health records (EHR) data.

After this module, students will be able to:

* Explain the lifecycle of typical machine learning (ML) projects.
* Preprocess datasets using Pandas implementations of common data pipelines.
* Train an XGBoost machine learning model using a structured patient dataset.
* Build a deep learning (DL) classifier using Keras.
* Explain model decision-making through interpretability techniques.

In this notebook, we will implement two machine learning models to predict the risk of acute kidney injury (AKI) following major inpatient surgery, using a tabular dataset of patient features.

### **🚀 Let's get started.**

# Example Application of Machine Learning in Critical Care

* Predicting surgical complications *before* surgery can inform (1) shared decisions regarding the appropriateness of procedures, (2) targeted risk-reduction strategies, and (3) postoperative resource use. 

* Cognitive and judgment errors are major sources of potentially preventable complications.
    * Example: if the risk of complications is underestimated, high-risk patients may be triaged to general wards rather than more appropriate intensive care units (ICUs).
    
* AI-based clinical decision support has the potential to mitigate harm from cognitive errors occurring when estimating the risk of postoperative complications.

* All patients have a unique risk profile that is specific to their demographic characteristics, comorbid conditions, physiological reserve, planned surgical procedure, and surgeon skill.

* Clinicians sometimes display moderate ability to estimate risk probabilities. AI tools are intended to augment decision-making.

* Many clinicians are hindered by time-consuming manual data entry requirements and lack of integration with clinical workflow.
    
* AI predictive models using automated EHR data inputs may be able to mitigate these challenges.

# Machine Learning Lifecycle
Shown below are the major steps that are involved in a typical ML lifecycle.
<center><img src="graphics/ml.png" alt="ML lifecycle" width="50%"></center>

### Data Extraction and Processing
* Relevant data must be extracted (example: all revelant laboratory values and vital signs).
* Data quality and conformation to modeling requirements must be ensured through data preprocessing (example: fixing the missing data, removing any unwanted outliers, and encoding the categorical data.)
  
### Model development
* Machine learning models are built from data.
* Recall our module on functions: an ML model can be thought of as a **function** that maps a set of input values (example: patient characteristics) to an output value (example: probability of developing AKI).
* We develop a model on data from a **training** cohort.
* During the model refinement stage, known as **training**, model parameters (such as *weights* and *biases*) are altered to optimize the associations between inputs and outputs.
* After training, a model is ready to make predictions on *new data it has not seen before*. But how accurate will it be?

### Evaluation and Intepretation
* Once a ML model is trained, we evaluate the end result on **testing** dataset to see how well the model performed or how reliable it is.
* Multiple performance metrics such as accuracy, AUC, sensitivity, and specificity can be evaluated.
* Once we are satisfied with a model's performance, we may implement **explainable AI (XAI)** techniques to discover **why** a model behaves in a particular way.
* XAI techniques include computing a relative importance score associated with each input feature/column (for a given patient, how important was each input in determining that patient's prediction?).

### Deployment
* ML model deployment is the process of placing a finished ML model into a live environment, such as creating a web service or mobile application for prediction.
* It is imperative that the performance of an established model is continually monitored in prospective deployment to safeguard against data distribution shifts resulting from changes in populations, practice, or other changes that may result in deteriorating model performance over time.

### Optimization
* A deployed ML model has real-world impact and must be continuously monitored and maintained.
* When new or updated data becomes available, or when we determine that an existing deployed model no longer results in satisfactory performance, we need to optimize the model by re-training and re-evaluating the ML model.

In this notebook, we will only focus on the first three phases, and save practical discussions on deployment and optimization for a later module.

# Example data description and problem definition

We will be using an EHR dataset containing 10,000 surgical patients.
* **Input features (X)** (47)
    * Demographic variables
    * Admission and surgery information
    * Social determinants of health (SDOH)
    * One-year medication history
    * Admission comorbidities
    * Estimated glomerular filtration rate (eGFR: measures kidney function) before surgery
* **Outcome (Y)** (1)
    * Postoperative AKI: onset of acute kidney injury (AKI) at any point between the end of surgery and patient discharge or death.
    * This is a binary outcome (it is recorded as either 0 or 1 for each patient). The ML model will learn to predict the risk of each patient developing AKI based on the specific input features.

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('https://www.dropbox.com/scl/fi/itlhhnrhbg9y9b3nnbd7t/data_unprocessed.csv?rlkey=labucmpbah1xu7alhmgrn4511&dl=1')
df.head()

,patient_id,sex,ethnicity,race,language,marital_status,age,bmi,admission_source,emergent,...,valve,coag,obese,wghtloss,lytes,anemia,alc_drug,depress,egfr,aki
0,0,MALE,NON-HISPANIC,WHITE,ENGLISH,MARRIED,62.4614,24.6711,NON-TRANSFER,EMERGENCY,...,0,0,1,0,0,0,0,1,82.1529,0
1,1,MALE,NON-HISPANIC,WHITE,ENGLISH,SINGLE,65.3903,30.5554,TRANSFER,NON-EMERGENCY,...,0,0,0,1,0,1,0,0,104.7827,1
2,2,FEMALE,NON-HISPANIC,WHITE,ENGLISH,SINGLE,82.6667,40.5842,NON-TRANSFER,NON-EMERGENCY,...,0,0,1,0,0,1,0,0,85.4759,0
3,3,MALE,NON-HISPANIC,WHITE,ENGLISH,MARRIED,62.1461,22.5506,NON-TRANSFER,EMERGENCY,...,0,1,1,0,1,0,1,0,57.2850,0
4,4,FEMALE,NON-HISPANIC,WHITE,ENGLISH,MARRIED,43.1582,16.7270,NON-TRANSFER,EMERGENCY,...,0,0,0,0,1,0,1,0,NaN,1


> As an exploratory step, let's take a look at the distribution of our AKI outcome (how prevalent is postoperative AKI?). We'll use the `value_counts` function we saw before to help answer this question.

In [2]:
# The normalize=True parameter divides each count by the total number of samples.
df['aki'].value_counts(normalize=True)

0    0.84
1    0.16
Name: aki, dtype: float64

> We can see from the above that about 16% of the surgical encounters resulted in acute kidney injury.

# Data Preprocessing

Before we use our data to train a machine learning model, we must transform the raw data into **features** that an ML model can understand (sometimes these look quite similar, but not always). This critical step is called **data preprocessing**. Since real-world data can be messy, during this phase we also *clean* data to ensure the ML model can learn meaningful patterns.

Common data preprocessing steps may include the following (depending on the ML model used):
* Removing outliers*.
* Addressing data missingess.
* Transforming binary string columns (e.g., sex) into a numerical form that the model can understand.
* Transforming many-valued string columns (e.g., race) into a numerical form.
* Normalizing input features to give them equal weighting.

\*One common approach to remove outliers in ML projects is to remove observations that fall outside a specified range. In this tutorial, we remove observations whose values are less than the 1st percentile or greater than the 99th percentile.


### Addressing missing data

* Missing data are common in routinely collected health data.
    * Many times, missingness can be informative (e.g. a particular test was not ordered because patient was healthy).
    * In other cases, missingness may be at random or due to other factors (e.g., lack of routine follow-up).
* Some models **require** filling in missing data with imputation methods. Others will gladly accept missing data. The decision of when, and how to impute missing data, is often dependent on the model, data, and problem. 
* In a Pandas DataFrame, missing values will be represented by `NaN` (*"Not a Number"*).

One simple, but effective, imputation approach is to replace a patient's missing variable value with a value computed from all other patients. For example, if a patient is missing the `bmi` variable, we might impute the mean value for the `bmi` feature across all other patients. (Recall our Pandas module for other choices!)

Let's use code to replace any missing `bmi` values with the `median` bmi from all other patients. We can accomplish this using some Pandas functions we've already seen, along with the `df.fillna(x)` function, which for a given DataFrame `df`, replaces any missing values with the input value `x`.

In [3]:
# We can use Pandas filtering to see examples of where bmi is missing. (Notice the NaN)
df[pd.isnull(df['bmi'])].head()

,patient_id,sex,ethnicity,race,language,marital_status,age,bmi,admission_source,emergent,...,valve,coag,obese,wghtloss,lytes,anemia,alc_drug,depress,egfr,aki
18,18,MALE,NON-HISPANIC,WHITE,ENGLISH,MARRIED,78.1377,NaN,TRANSFER,NON-EMERGENCY,...,0,1,1,0,1,0,0,0,NaN,0
22,22,FEMALE,NON-HISPANIC,WHITE,ENGLISH,DIVORCED,31.9612,NaN,TRANSFER,NON-EMERGENCY,...,0,1,1,0,1,0,0,0,112.9666,0
125,125,MALE,HISPANIC,WHITE,ENGLISH,MARRIED,78.8370,NaN,NON-TRANSFER,NON-EMERGENCY,...,0,1,1,0,1,0,1,0,139.9804,0
140,140,MALE,NON-HISPANIC,WHITE,ENGLISH,MARRIED,66.4866,NaN,NON-TRANSFER,EMERGENCY,...,1,0,0,0,1,1,0,1,83.5386,0
151,151,MALE,NON-HISPANIC,WHITE,NON-ENGLISH,MARRIED,33.9240,NaN,NON-TRANSFER,EMERGENCY,...,1,0,1,0,1,0,0,0,NaN,1


In [4]:
# Another way to see missing BMI values
df[df['bmi'].isna()].head()

,patient_id,sex,ethnicity,race,language,marital_status,age,bmi,admission_source,emergent,...,valve,coag,obese,wghtloss,lytes,anemia,alc_drug,depress,egfr,aki
18,18,MALE,NON-HISPANIC,WHITE,ENGLISH,MARRIED,78.1377,NaN,TRANSFER,NON-EMERGENCY,...,0,1,1,0,1,0,0,0,NaN,0
22,22,FEMALE,NON-HISPANIC,WHITE,ENGLISH,DIVORCED,31.9612,NaN,TRANSFER,NON-EMERGENCY,...,0,1,1,0,1,0,0,0,112.9666,0
125,125,MALE,HISPANIC,WHITE,ENGLISH,MARRIED,78.8370,NaN,NON-TRANSFER,NON-EMERGENCY,...,0,1,1,0,1,0,1,0,139.9804,0
140,140,MALE,NON-HISPANIC,WHITE,ENGLISH,MARRIED,66.4866,NaN,NON-TRANSFER,EMERGENCY,...,1,0,0,0,1,1,0,1,83.5386,0
151,151,MALE,NON-HISPANIC,WHITE,NON-ENGLISH,MARRIED,33.9240,NaN,NON-TRANSFER,EMERGENCY,...,1,0,1,0,1,0,0,0,NaN,1


In [5]:
# Compute the median BMI (note: missing values are ignored in this calculation)
bmi_median = df['bmi'].median()
print(bmi_median)

28.878349999999998


In [6]:
# Use fillna to replace any missing BMI values with the median BMI value.
df['bmi'] = df['bmi'].fillna(bmi_median)

In [7]:
# Let's re-run this code to see where BMI is still missing
df[pd.isnull(df['bmi'])]

,patient_id,sex,ethnicity,race,language,marital_status,age,bmi,admission_source,emergent,...,valve,coag,obese,wghtloss,lytes,anemia,alc_drug,depress,egfr,aki


**BMI is no longer missing!** Every patient now has a BMI value for passing into the model.

> **📌 Note:** There are many alternative approaches for missing data imputation, including k-Nearest neighbors (KNN) and multiple imputation by chained equation ([MICE](https://www.jstatsoft.org/article/view/v045i03)).

# Data Transformation
In general, machine learning models are great at learning from **numerical** data (e.g., 0, 1, 0.5) . However, many important clinical characteristics are stored as **categorical** data (e.g., `MALE`, `DIVORCED`). For many models, we must convert the categorical values to numerical features through a **data transformation** process.

For categorical variables with **only two** options, we can assign one of the options to be **0**, and the other to be **1**.

Let's see an example of how we would do this for the `sex` variable, which in our particular dataset contains only two values: `MALE` and `FEMALE`:

In [8]:
# Show the unique values of the sex variable before transformation
print(df['sex'].unique())

# We can use the replace function to convert strings to binary numbers
df['sex'] = df['sex'].replace({'FEMALE' : 0, 'MALE' : 1})

# Show the unique values of the sex variable after transformation
print(df['sex'].unique())

['MALE' 'FEMALE']
[1 0]


Perfect! But what if our categorical variable contains more than two unique values?
* One option would be to replace them with more numbers (for example: 0, 1, 2, 3, ...)
* However, doing this may confuse some models due to implicit ordering and hierarchies
    * If 0=apple, 1=banana, 2=pear, 3=tomato, then does a tomato equal three bananas? Is a pear worth more than a banana, which is worth more than an apple? 

One solution is called **one-hot encoding**:
1. For a single categorical variable (e.g., `marital_status`), create a separate column in the dataset for each category of a categorical variable.
    * If there are 4 options, then we create 4 new columns.
2. The values of these columns are zero **except for** the column corresponding to the original value.

Let's see this in action:

In [9]:
# Show the first 5 values of marital_status before one-hot encoding
print(df['marital_status'].head())

# Use the get_dummies() function to perform one-hot encoding on the marital_status column.
marital_status_transformed = pd.get_dummies(df['marital_status'])

# Show the first 5 values of marital_status after one-hot encoding
print(marital_status_transformed.head())

0    MARRIED
1     SINGLE
2     SINGLE
3    MARRIED
4    MARRIED
Name: marital_status, dtype: object
   DIVORCED  MARRIED  MISSING  SINGLE
0         0        1        0       0
1         0        0        0       1
2         0        0        0       1
3         0        1        0       0
4         0        1        0       0


# Let's Skip Ahead.
Data analysis, imputation, transformation, and preprocessing take a large portion of any AI/ML project's time. We only covered a few steps of the process in this module. Later modules will explore more advanced methods to manipulate and transform patient data into AI/ML-ready formats.

🕑 For now, let's save time by loading a new dataset that has already been preprocessed:

In [10]:
df = pd.read_csv('https://www.dropbox.com/scl/fi/zzsy0dlbwn8vqk9vaqaar/data_processed.csv?rlkey=05u63ywmemb3ubu9kday4p23g&dl=1')
df.head()

,patient_id,sex,ethnicity,race,language,marital_status,age,bmi,admission_source,emergent,...,valve,coag,obese,wghtloss,lytes,anemia,alc_drug,depress,egfr,aki
0,0,0.978237,0.263234,-0.203467,0.136533,-1.017430,-0.684920,-0.833419,-0.441135,-0.761596,...,-0.445011,-0.382959,-0.849218,-0.420578,-0.749675,-0.567958,2.357409,-0.645470,0.783296,0
1,1,-1.022247,0.263234,2.315985,0.136533,-1.017430,-1.259334,-0.523094,-0.441135,-0.761596,...,-0.445011,-0.382959,-0.849218,-0.420578,-0.749675,-0.567958,-0.424195,1.549259,0.052083,0
2,2,-1.022247,0.263234,-0.203467,0.136533,0.144422,1.038322,0.377905,-0.441135,1.313032,...,-0.445011,-0.382959,1.177555,-0.420578,-0.749675,-0.567958,-0.424195,1.549259,-0.357553,0
3,3,-1.022247,0.263234,2.315985,0.136533,0.144422,0.463908,3.470699,-0.441135,1.313032,...,2.247133,2.611247,1.177555,-0.420578,1.333912,1.760693,-0.424195,1.549259,-0.789798,0
4,4,0.978237,0.263234,-0.203467,0.136533,0.144422,0.751115,0.013019,-0.441135,-0.761596,...,2.247133,-0.382959,1.177555,-0.420578,1.333912,-0.567958,-0.424195,1.549259,-1.845950,1


# Model development

It's now time to train our first machine learning model!

We're going to develop our very own **XGBoost** model to predict postoperative AKI from our patient characteristics contained in `df`. XGBoost is a popular tree-based model that typically works well with tabular datasets like ours (and usually requires minimal tinkering).

<center><img src="https://media.licdn.com/dms/image/C4D12AQHIt2funjnANA/article-cover_image-shrink_720_1280/0/1623913883834?e=1718841600&v=beta&t=QSFJ5MO4hvNEsAT5rB7KWxRwDwf28yMnUQjhQUUYpb0" /></center>

# Splitting the dataset

* **Recall** that ML models adjust their internal parameters by feeding in many examples of inputs $X$ and corresponding output $y$. This is called the **training** process. With enough time and training examples, the model can learn associations between inputs and outputs that can help it make a prediction on new data it hasn't ever seen before.

* To evaluate how good our ML model is, we will **test** it on a set of examples that it has never seen before (known as the **test set**), and for each patient in the test set, compare the model's predicted outcome $\hat{y}$ with the true patient outcome $y$.

* With enough testing examples, we can calculate performance metrics such as accuracy and other diagnostic metrics like AUC, sensitivity, and specificity.

Let's take our dataset and prepare it for building our ML model by doing a few final preprocessing items:
* Convert the pandas DataFrame into two NumPy arrays: `X` (inputs) and `y` (AKI outcome)
* Split X and y into training and testing sets. We'll have four variables after this step:
    * `X_train` and `y_train`: the training set
    * `X_test` and `y_test`: the testing set
    * We'll arbitrarily use 20% of the dataset for testing.

> **📌 Note:** Some of this code may seem unfamiliar to you. That's OK! Follow along with the comments to understand each step.

In [11]:
from sklearn.model_selection import train_test_split # We'll use this function to create the training and testing sets

# Build X (an array/matrix) by taking the values from the entire df DataFrame EXCEPT FOR the first (patient id) and last (AKI outcome) columns
X = df.iloc[:, 1:-1].values

# Build y (a one-dimensional array) by taking only the final column ('aki'). The order of these outcomes exactly matches the order of the rows in X.
y = df.iloc[:, -1].values

# Split X and y into training and testing sets, by using a random 20% of the dataset for testing.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=2024)

# Creating and Training an XGBoost Model

* Let's train our first XGBoost model to predict postoperative AKI from our patient features.
* We're going to use the popular Python library `xgboost`, which makes it very easy to train an ML model ***in a single line of code!***

In [12]:
import xgboost as xgb

# Create our XGBoost prediction model and assign it to variable xgb
xgb = xgb.XGBClassifier()

# Train the model using the training dataset
xgb.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

That was easy! But is this model any good?

To evaluate the performance of our model, we need to compare two quantities:  
* $y$, the true outcomes of our testing set patients. In our code, we'll call this list `y_true`.
* $\hat{y}$, our model's **predicted** outcomes for the testing set patients. We'll call this `y_pred` (predicted).

The XGBoost model has an easy function called `score` that computes the accuracy on a test dataset.

In [13]:
# By default, xgb.score() will compute the model's accuracy on a given dataset.
xgb.score(X_test, y_test)

0.825

In [14]:
# We can also compute the area under the receiver operating characteristic curve with scikit-learn's roc_auc_score() function.
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, xgb.predict_proba(X_test)[:, 1])

0.7278645833333334

# Feature Importance
* We now have our trained XGBoost model, and know how accurate it is at making predictions, but we don't understand **why** it predicts certain outcomes for certain patients.
* For this, we need to turn to the field of **explainable AI (XAI)**, an emerging and active field of AI research.
* In this module, we'll examine one of the simplest forms of explainability, which looks at the model's weight it assigns to each feature.
* Every XGBoost model has a built-in *attribute* variable for this: `feature_importances_` (for convenience, we'll reassign the feature importances to a variable `F`).

In [15]:
# Compute feature importances
F = xgb.feature_importances_

# Take a look at the feature importances.
# The scores are in the order of the columns of our input data.
print(F)

[0.01526557 0.01500232 0.02121888 0.         0.02391919 0.01852124
 0.01908465 0.03381221 0.02769355 0.05297443 0.02130473 0.02169216
 0.01892946 0.01547223 0.01722141 0.02002565 0.00646832 0.00209006
 0.00940535 0.00543704 0.02706463 0.01127416 0.02406041 0.02466714
 0.01574473 0.06533682 0.01108566 0.06463631 0.00967965 0.04050528
 0.01108665 0.0122027  0.         0.02290084 0.05108256 0.02290248
 0.02213632 0.02852025 0.01375859 0.00805862 0.01885648 0.02076913
 0.02797993 0.02480106 0.01851321 0.00881022 0.02802767]


To make this more informative, we can map these scores to their original feature names and generate a plot to visually compare the importance of each feature.

Let's do this for the model's top 10 features.

> **📌 Note:** Some of this code will be new. It's OK to just follow along!

<Axes: ylabel='feature'>

In [16]:
F_df = pd.DataFrame({'feature': df.columns.tolist()[1:-1], 'score': F}).sort_values(by='score').tail(10)
F_df.plot.barh(x='feature', y='score', figsize=(5,3))

By examining the model's learned feature importance scores, we can better understand influential model inputs and begin to have an idea of what information the model is weighting most heavily when it makes its predictions.

---
> Portions of this module are based on concepts from course materials from BME4760 Biomedical Data Science, Department of Biomedical Engineering, University of Florida.
---